In [4]:
# Image Classification
import torch
import numpy as np
import torchvision.transforms as T

H, W = 32, 32
# img = torch.randint(0, 256, size=(3, H, W), dtype=torch.uint8)
img = np.random.randint(0, 256, size=(3, H, W), dtype=np.uint8)

transforms = T.Compose([
    T.RandomResizedCrop(size=(224, 224), antialias=True),
    T.RandomHorizontalFlip(p=0.5),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
img = transforms(img)

TypeError: Unexpected type <class 'numpy.ndarray'>